In [7]:
def GetP(sheet_id='11u2eirdZcWZdjnKFn3vzqbCtKCodstP-KnoGXC5FdR8', 
             sheet_name='378514385'):
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&gid={sheet_name}"
    p = pd.read_csv(url)
    p = p.dropna(axis=1, how='all')
    for i in range(len(p)):
        try:
            if np.isnan(p[p.columns[0]][0]):
                p = p.drop(i, axis=0)
        except TypeError:
            pass
    return p

p = GetP(sheet_name='378514385')
BaseModelp = GetP(sheet_name='0')
p

,opa_file,wave_range,calculation,R,fsed,kzz,mmw,condensates,virga_mieff,output_dir
0,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,0.1,1.000000e+09,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
1,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,0.1,1.000000e+11,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
2,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,0.5,1.000000e+09,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
3,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,0.5,1.000000e+11,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
4,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,1.0,1.000000e+09,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
5,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,1.0,1.000000e+11,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
6,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,3.0,1.000000e+09,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
7,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,3.0,1.000000e+11,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
8,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,6.0,1.000000e+09,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy
9,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000,6.0,1.000000e+11,2.2,virga recommend,/data/virga,/data2/lpearce/ReflectXGasGiantModelGrid/cloudy


In [5]:
def Run1CloudyModel(ps, num_tangle = 6, num_gangle = 6):
    OneCloudp, OneBaseModelp = ps
    ### Load basemodel:
    p = OneBaseModelp
    grid = p['name']
    # Planet:
    planettype = p['planet_type']
    Tint = p['tint'] # Internal Temperature of your Planet in K
    Teq = ComputeTeq(p['st_teff'], p['rstar']*u.Rsun, p['au']*u.au, 
                     Ab = 0.3, fprime = 1/4) # planet equilibrium temperature 
    radius = p['pl_rad'] #Rjup
    massj = p['pl_mass']
    semi_major = p['au']
    phase = p['phase']

    # Star:
    T_star = p['st_teff'] # K, star effective temperature
    logg = p['logg'] #logg , cgs
    metal = p['feh'] # metallicity of star
    r_star = p['rstar'] # solar radius

    Teq_str = np.round(Teq, decimals=0)
    directory = f'{grid}-{planettype}-Tstar{int(T_star)}-Rstar{r_star}-Teq{int(Teq_str)}-sep{semi_major}-rad{radius}-mass{massj}-mh{int(planet_mh)}-co{planet_mh_CtoO}-phase{int(phase)}'
    #savefiledirectory = p['output_dir']+directory
    output_dir = ''
    savefiledirectory = output_dir+directory
    
    fsed = Cloudp['fsed']
    logkzz = np.log10(Cloudp['kzz'])
    cloud_filename_prefix = f'-fsed{fsed}-kzz{int(logkzz)}'
    
    
    ### Cloud setup:
    clouds_setup = {'kz':OneCloudp['kzz'], 
                    'fsed':OneCloudp['fsed'], 
                    'mmw':OneCloudp['mmw'],
                    'condensates':OneCloudp['condensates'], 
                    'virga_mieff':OneCloudp['virga_mieff']
                   }
    
    ## Spectrum setup
    opa_file = None
    #opa_file = Cloudp['opa_file']
    #R = Cloudp['R']
    R = 150
    wave_range = [float(OneCloudp['wave_range'].split(',')[0].replace('[','')),
              float(OneCloudp['wave_range'].split(',')[1].replace(' ','').replace(']',''))]
    spectrum_setup = {'opacity_db':opa_file,
                      'wave_range':wave_range,
                      'calculation':'reflected', 'R':R
                     }
   
    
    clouds, clouds_added = MakeModelCloudyPlanet(savefiledirectory, 
                                                 cloud_setup, 
                                                 spectrum_setup,
                                                 cloud_filename_prefix,
                                                 calculation = 'planet')
    
    
    
    

,name,planet_type,output_dir,st_teff,rstar,feh,logg,pl_mass,pl_rad,mh,...,nstr_deep,nofczns,p_bottom,p_top,guess,local_ck_path,opa_file,wave_range,calculation,R
0,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
1,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
2,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
3,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
4,ReflectXGasGiantModelGrid,Neptune,/data2/lpearce/ReflectXGasGiantModelGrid/,3000.0,0.2,1.0,5.0,0.05,0.35,100.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,ReflectXGasGiantModelGrid,SuperJup,/data2/lpearce/ReflectXGasGiantModelGrid/,7000.0,10.0,1.0,2.6,5.00,0.90,1.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
1246,ReflectXGasGiantModelGrid,SuperJup,/data2/lpearce/ReflectXGasGiantModelGrid/,7000.0,10.0,1.0,2.6,5.00,0.90,1.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
1247,ReflectXGasGiantModelGrid,SuperJup,/data2/lpearce/ReflectXGasGiantModelGrid/,7000.0,10.0,1.0,2.6,5.00,0.90,1.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0
1248,ReflectXGasGiantModelGrid,SuperJup,/data2/lpearce/ReflectXGasGiantModelGrid/,7000.0,10.0,1.0,2.6,5.00,0.90,1.0,...,89.0,1.0,2.0,-6.0,guillot,/data/kcoeff_2020_v3/,/data2/picaso_dbs/R60000/all_opacities_0.3_2_R...,"[0.4, 2.0]",Reflected,2000.0


In [3]:
sheet_id='11u2eirdZcWZdjnKFn3vzqbCtKCodstP-KnoGXC5FdR8'
sheet_name='378514385'
CloudyP = GetP(sheet_id=sheet_id, sheet_name='378514385')
BaseModelP = GetP(sheet_id=sheet_id, sheet_name='0')
ps = [CloudyP, BaseModelP]

In [6]:
CloudyP[CloudyP.columns[0]][0]

'/data2/picaso_dbs/R60000/all_opacities_0.3_2_R60000.db'